In [3]:
# 1) Kütüphaneler
!pip install statsmodels scikit-learn matplotlib seaborn tqdm --quiet

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.api as sm
from google.colab import files

sns.set(context="notebook", style="whitegrid", rc={"figure.dpi":130})
plt.rcParams["figure.figsize"] = (9,6)
print("Kütüphaneler yüklendi ve içe aktarıldı.")

Kütüphaneler yüklendi ve içe aktarıldı.


In [4]:
# 2) Dosya yükleme (Colab) - çalıştırınca pencere açılır
print("CSV dosyalarını seç: WDICSV.csv, WDICountry.csv, WDISeries.csv, WDIcountry-series.csv, WDIfootnote.csv, WDIseries-time.csv")
uploaded = files.upload()

# 3) Dosyaları oku (isimler aynıysa)
wdi_main = pd.read_csv("WDICSV.csv", low_memory=False)
wdi_country = pd.read_csv("WDICountry.csv")
wdi_series = pd.read_csv("WDISeries.csv")
# opsiyonel/destek
try:
    wdi_country_series = pd.read_csv("WDIcountry-series.csv")
except:
    wdi_country_series = None
try:
    wdi_footnote = pd.read_csv("WDIfootnote.csv")
except:
    wdi_footnote = None
try:
    wdi_series_time = pd.read_csv("WDIseries-time.csv")
except:
    wdi_series_time = None

print("Dosyalar okundu:", wdi_main.shape, wdi_country.shape, wdi_series.shape)

CSV dosyalarını seç: WDICSV.csv, WDICountry.csv, WDISeries.csv, WDIcountry-series.csv, WDIfootnote.csv, WDIseries-time.csv


Saving WDICountry.csv to WDICountry.csv
Saving WDIcountry-series.csv to WDIcountry-series.csv
Saving WDICSV.csv to WDICSV (1).csv
Saving WDIfootnote.csv to WDIfootnote.csv
Saving WDISeries.csv to WDISeries.csv
Saving WDIseries-time.csv to WDIseries-time.csv
Dosyalar okundu: (403256, 69) (265, 31) (1516, 20)


In [5]:
# 4) Uzun formata çevir (WDICSV tipik format: yıllar sütun)
# Bulunan yıl sütunlarını tespit et
year_cols = [c for c in wdi_main.columns if c.strip().isdigit()]
id_vars = [c for c in wdi_main.columns if c not in year_cols]

wdi_long = wdi_main.melt(id_vars=id_vars, value_vars=year_cols,
                         var_name="Year", value_name="Value")

# temiz ve tip düzelt
wdi_long['Year'] = wdi_long['Year'].astype(int)
# Bazı WDI dosyalarında ".." gibi notlar olur:
wdi_long = wdi_long[wdi_long['Value'].notna()]
wdi_long = wdi_long[wdi_long['Value'] != ".."]
wdi_long['Value'] = pd.to_numeric(wdi_long['Value'], errors='coerce')
wdi_long.dropna(subset=['Value'], inplace=True)

# normalize column names if different (defensive)
for col in wdi_long.columns:
    # nothing
    pass

print("Veri uzun formata çevrildi ve ilk temizlik yapıldı.")

Veri uzun formata çevrildi ve ilk temizlik yapıldı.


In [6]:
# 5) Sadece hedef göstergeleri seç: GDP per capita & CO2 per capita
# Normalize names
wdi_long.rename(columns=lambda x: x.strip(), inplace=True)
# Expecting: 'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'Year', 'Value'
# If different, try some safe mapping:
cols = wdi_long.columns.tolist()
# Mapping (if present)
if 'Country Name' not in cols and 'Country' in cols:
    wdi_long.rename(columns={'Country': 'Country Name'}, inplace=True)
if 'Indicator Code' not in cols:
    for c in cols:
        if c.lower().startswith('indicator') and 'code' in c.lower():
            wdi_long.rename(columns={c: 'Indicator Code'}, inplace=True)
if 'Indicator Name' not in cols:
    for c in cols:
        if c.lower().startswith('indicator') and 'name' in c.lower():
            wdi_long.rename(columns={c: 'Indicator Name'}, inplace=True)
if 'Country Code' not in cols:
    for c in cols:
        if 'country code' in c.lower() or c.lower()=='countrycode':
            wdi_long.rename(columns={c:'Country Code'}, inplace=True)

# 5.1 target indicators
GDP_CODE = 'NY.GDP.PCAP.CD'
CO2_CODE = 'EN.ATM.CO2E.PC'

target = wdi_long[wdi_long['Indicator Code'].isin([GDP_CODE, CO2_CODE])].copy()
print(f"Hedef göstergeler seçildi. Satır sayısı: {target.shape[0]}")

Hedef göstergeler seçildi. Satır sayısı: 14547


In [25]:
# 6️⃣ GDP ve CO2 göstergelerini belirle ve birleştir

# CO₂ kişi başına göstergesini kontrol et
unique_codes = wdi_long['Indicator Code'].unique()
print("WDI veri setindeki mevcut CO₂ göstergeleri:")
print([code for code in unique_codes if 'CO2' in code.upper() or 'CO₂' in code.upper()])

# CO₂ göstergesi seçim mantığı
if 'EN.ATM.CO2E.PC' in unique_codes:
    CO2_CODE = 'EN.ATM.CO2E.PC'
    print("Kişi başına CO₂: EN.ATM.CO2E.PC kullanılıyor.")
elif 'EN.GHG.CO2.PC.CE.AR5' in unique_codes:
    CO2_CODE = 'EN.GHG.CO2.PC.CE.AR5'
    print("Kişi başına CO₂: EN.GHG.CO2.PC.CE.AR5 kullanılıyor.")
elif 'EN.ATM.CO2E.KT' in unique_codes:
    print("Kişi başına CO₂ yok, toplam CO₂ ve nüfustan hesaplanacak...")
    CO2_CODE = None
else:
    raise ValueError("Uygun CO₂ göstergesi bulunamadı!")

# CO₂ kişi başına veriyi oluştur
if CO2_CODE:
    co2_data = wdi_long[wdi_long['Indicator Code'] == CO2_CODE][['Country Name','Year','Value']]
    co2_data.rename(columns={'Value':'co2_per_capita'}, inplace=True)
else:
    # Nüfus ve toplam CO2 verisinden kişi başına hesaplama
    POP_CODE = 'SP.POP.TOTL'
    pop_data = wdi_long[wdi_long['Indicator Code'] == POP_CODE][['Country Name','Year','Value']]
    pop_data.rename(columns={'Value':'population'}, inplace=True)

    co2_data = wdi_long[wdi_long['Indicator Code'] == 'EN.ATM.CO2E.KT'][['Country Name','Year','Value']]
    co2_data.rename(columns={'Value':'co2_total_kt'}, inplace=True)

    co2_data = co2_data.merge(pop_data, on=['Country Name','Year'], how='inner')
    co2_data['co2_per_capita'] = (co2_data['co2_total_kt'] * 1e3) / co2_data['population']

# GDP kişi başına veriyi çek
gdp_data = wdi_long[wdi_long['Indicator Code'] == 'NY.GDP.PCAP.CD'][['Country Name','Year','Value']]
gdp_data.rename(columns={'Value':'gdp_per_capita'}, inplace=True)

# GDP ve CO2 kişi başına verilerini birleştir
pivot = gdp_data.merge(co2_data[['Country Name','Year','co2_per_capita']], on=['Country Name','Year'], how='inner')
print("GDP ve CO₂ kişi başı veriler birleştirildi. Pivot tablo hazır.")
print("Pivot kolonları:", pivot.columns.tolist())

# 7️⃣ En az %80 oranında dolu olan ülkeleri seç
valid_countries = []
grouped = pivot.groupby('Country Name')
for country, g in grouped:
    both_nonnull = g[['gdp_per_capita','co2_per_capita']].notna().all(axis=1).mean()
    if both_nonnull >= 0.8:
        valid_countries.append(country)

print(f"Ülke sayısı (>=%80 doluluk): {len(valid_countries)}")
print("Geçerli ülkelerden ilk 10 örnek:", valid_countries[:10])


WDI veri setindeki mevcut CO₂ göstergeleri:
['EN.GHG.CO2.MT.CE.AR5', 'EN.GHG.CO2.PC.CE.AR5', 'EN.GHG.CO2.AG.MT.CE.AR5', 'EN.GHG.CO2.BU.MT.CE.AR5', 'EN.GHG.CO2.FE.MT.CE.AR5', 'EN.GHG.CO2.IC.MT.CE.AR5', 'EN.GHG.CO2.IP.MT.CE.AR5', 'EN.GHG.CO2.PI.MT.CE.AR5', 'EN.GHG.CO2.TR.MT.CE.AR5', 'EN.GHG.CO2.WA.MT.CE.AR5', 'EN.GHG.CO2.RT.GDP.KD', 'NY.ADJ.DCO2.CD', 'NY.ADJ.DCO2.GN.ZS', 'EN.GHG.CO2.RT.GDP.PP.KD', 'EN.GHG.CO2.LU.DF.MT.CE.AR5', 'EN.GHG.CO2.LU.FL.MT.CE.AR5', 'EN.GHG.CO2.LU.OS.MT.CE.AR5', 'EN.GHG.CO2.LU.OL.MT.CE.AR5', 'EN.GHG.CO2.ZG.AR5', 'EN.GHG.CO2.LU.MT.CE.AR5', 'IC.FRM.CO2.ZS']
Kişi başına CO₂: EN.GHG.CO2.PC.CE.AR5 kullanılıyor.
GDP ve CO₂ kişi başı veriler birleştirildi. Pivot tablo hazır.
Pivot kolonları: ['Country Name', 'Year', 'gdp_per_capita', 'co2_per_capita']
Ülke sayısı (>=%80 doluluk): 248
Geçerli ülkelerden ilk 10 örnek: ['Afghanistan', 'Africa Eastern and Southern', 'Africa Western and Central', 'Albania', 'Algeria', 'American Samoa', 'Angola', 'Antigua and Barbuda', 'Arab W

In [28]:
# 8) %80 doluluk: ülke bazında (gdp & co2 ikilisine bak)
valid_countries = []
grouped = pivot.groupby('Country Name')
for country, g in grouped:
    # oran: her iki sütunun da non-null oranı
    both_nonnull = g[['gdp_per_capita','co2_per_capita']].notna().all(axis=1).mean()
    if both_nonnull >= 0.8:
        valid_countries.append(country)

pivot = pivot[pivot['Country Name'].isin(valid_countries)].copy()
print(f"Ülke sayısı (>=%80 doluluk): {pivot['Country Name'].nunique()}")

# 9) Ek temizlik:
pivot = pivot[pivot['gdp_per_capita']>0]

low_q = pivot['co2_per_capita'].quantile(0.01)
high_q = pivot['co2_per_capita'].quantile(0.99)
pivot['co2_wins'] = pivot['co2_per_capita'].clip(lower=low_q, upper=high_q)

pivot['log_gdp_per_capita'] = np.log(pivot['gdp_per_capita'])
print("Veri hazırlığı (log, winsorize) tamamlandı.")

Ülke sayısı (>=%80 doluluk): 248
Veri hazırlığı (log, winsorize) tamamlandı.


In [30]:
# 10) Basit EDA ve meta veri birleştirme (güncellenmiş)
print("WDI ülke meta bilgileri birleştiriliyor...")

# Kolon isimlerini kontrol et
print("WDI country kolonları:", wdi_country.columns.tolist())

# Uygun eşleştirme sütunu belirle
if 'Country Name' in pivot.columns and 'Table Name' in wdi_country.columns:
    pivot = pivot.merge(wdi_country, left_on='Country Name', right_on='Table Name', how='left', suffixes=('', '_meta'))
    print("✅ Ülke meta verileri (bölge, gelir grubu vb.) ana tabloya eklendi.")
else:
    print("⚠️ Eşleştirme başarısız: pivot veya wdi_country içinde gerekli kolonlar bulunamadı.")



WDI ülke meta bilgileri birleştiriliyor...
WDI country kolonları: ['Country Code', 'Short Name', 'Table Name', 'Long Name', '2-alpha code', 'Currency Unit', 'Special Notes', 'Region', 'Income Group', 'WB-2 code', 'National accounts base year', 'National accounts reference year', 'SNA price valuation', 'Lending category', 'Other groups', 'System of National Accounts', 'Alternative conversion factor', 'PPP survey year', 'Balance of Payments Manual in use', 'External debt Reporting status', 'System of trade', 'Government Accounting concept', 'IMF data dissemination standard', 'Latest population census', 'Latest household survey', 'Source of most recent Income and expenditure data', 'Vital registration complete', 'Latest agricultural census', 'Latest industrial data', 'Latest trade data', 'Latest water withdrawal data']
✅ Ülke meta verileri (bölge, gelir grubu vb.) ana tabloya eklendi.


In [31]:
# 11) EKC modelleme - two approaches:
# A) Statsmodels OLS polynomial (degree 2): co2 ~ log_gdp + (log_gdp)^2

# Prepare data for modeling (drop NA)
model_df = pivot[['Country Name','Year','log_gdp_per_capita','co2_wins']].dropna().copy()

# A) OLS (pooled cross-section/time) - note: this is a simple pooled OLS
model_df['log_gdp_sq'] = model_df['log_gdp_per_capita']**2
X_ols = sm.add_constant(model_df[['log_gdp_per_capita','log_gdp_sq']])
y = model_df['co2_wins']
ols_model = sm.OLS(y, X_ols).fit(cov_type='HC3')  # robust SEs
print("\n--- OLS (pooled) summary ---")
print(ols_model.summary())


--- OLS (pooled) summary ---
                            OLS Regression Results                            
Dep. Variable:               co2_wins   R-squared:                       0.323
Model:                            OLS   Adj. R-squared:                  0.323
Method:                 Least Squares   F-statistic:                     2744.
Date:                Fri, 31 Oct 2025   Prob (F-statistic):               0.00
Time:                        18:20:43   Log-Likelihood:                -37805.
No. Observations:               12309   AIC:                         7.562e+04
Df Residuals:                   12306   BIC:                         7.564e+04
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const 

In [32]:
# B) Polynomial + Ridge with CV (performs regularization & CV)
X = model_df[['log_gdp_per_capita']].values
y = model_df['co2_wins'].values

def rmse_cv(estimator, X, y):
    # using 5-fold time-shuffled CV (pooled)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    neg_mse = cross_val_score(estimator, X, y, scoring='neg_mean_squared_error', cv=kf)
    return np.sqrt(-neg_mse.mean())

# try degrees and alphas
degrees = [1,2,3]
alphas = [0.0, 0.1, 1.0, 10.0, 100.0]  # alpha=0 uses Ridge with alpha=0 -> OLS equivalent
best = {'score':np.inf}
for deg in degrees:
    for a in alphas:
        steps = []
        steps.append(('poly', PolynomialFeatures(degree=deg, include_bias=False)))
        steps.append(('ridge', Ridge(alpha=a, random_state=42))) # random_state eklendi
        pipe = Pipeline(steps)
        score = rmse_cv(pipe, X, y)
        if score < best['score']:
            best = {'degree':deg, 'alpha':a, 'score':score, 'pipe':pipe}
        print(f"deg={deg} alpha={a} RMSE={score:.4f}")

print("\nBest model (sklearn pipeline):", best)

# Fit best pipeline on full data
best_pipe = best['pipe']
best_pipe.fit(X,y)
print("En iyi model tüm veri üzerinde eğitildi.")

deg=1 alpha=0.0 RMSE=5.2633
deg=1 alpha=0.1 RMSE=5.2633
deg=1 alpha=1.0 RMSE=5.2633
deg=1 alpha=10.0 RMSE=5.2633
deg=1 alpha=100.0 RMSE=5.2633
deg=2 alpha=0.0 RMSE=5.2200
deg=2 alpha=0.1 RMSE=5.2200
deg=2 alpha=1.0 RMSE=5.2200
deg=2 alpha=10.0 RMSE=5.2200
deg=2 alpha=100.0 RMSE=5.2205
deg=3 alpha=0.0 RMSE=5.1948
deg=3 alpha=0.1 RMSE=5.1948
deg=3 alpha=1.0 RMSE=5.1951
deg=3 alpha=10.0 RMSE=5.2024
deg=3 alpha=100.0 RMSE=5.2194

Best model (sklearn pipeline): {'degree': 3, 'alpha': 0.1, 'score': np.float64(5.194844016192656), 'pipe': Pipeline(steps=[('poly', PolynomialFeatures(degree=3, include_bias=False)),
                ('ridge', Ridge(alpha=0.1, random_state=42))])}
En iyi model tüm veri üzerinde eğitildi.


In [34]:
# 12) Model interpret & country-level residuals
# Tahminleri yaparken .values kullanarak uyarıyı kaldır
model_df['pred_ridge'] = best_pipe.predict(model_df[['log_gdp_per_capita']].values)
model_df['residual'] = model_df['co2_wins'] - model_df['pred_ridge']

# Aggregate for countries: check trend 2000->2020 for green growth candidate detection
agg2000 = pivot[pivot['Year']==2000].set_index('Country Name')[['gdp_per_capita','co2_wins']]
agg2020 = pivot[pivot['Year']==2020].set_index('Country Name')[['gdp_per_capita','co2_wins']]

agg = pd.DataFrame(index=agg2000.index)
agg['gdp2000'] = agg2000['gdp_per_capita']
agg['gdp2020'] = agg2020['gdp_per_capita']
agg['co22000'] = agg2000['co2_wins']
agg['co22020'] = agg2020['co2_wins']
agg = agg.dropna()

agg['gdp_change'] = agg['gdp2020'] - agg['gdp2000']
agg['co2_change'] = agg['co22020'] - agg['co22000']

# Yeşil büyüme adayları: GDP artıyor, CO2 düşüyor
green_candidates = agg[(agg['gdp_change']>0) & (agg['co2_change']<0)].copy()
green_candidates = green_candidates.sort_values('gdp_change', ascending=False)

print("\nYeşil büyüme adayları (örnek):")
print(green_candidates.head())



Yeşil büyüme adayları (örnek):
                   gdp2000        gdp2020    co22000    co22020    gdp_change  \
Country Name                                                                    
Luxembourg    48659.598875  116860.028172  20.171212  12.604157  68200.429297   
Ireland       26334.567205   86622.506725  11.516609   6.647496  60287.939520   
Switzerland   38865.021940   85897.784334   6.235849   4.154446  47032.762394   
Singapore     23852.838951   61410.079263  11.301037   9.476104  37557.240312   
Denmark       30722.025184   60985.488560   9.954986   4.766725  30263.463376   

              co2_change  
Country Name              
Luxembourg     -7.567056  
Ireland        -4.869112  
Switzerland    -2.081403  
Singapore      -1.824933  
Denmark        -5.188260  


In [35]:
# 13) Görseller: 3 etkili görsel
os.makedirs("outputs", exist_ok=True)

# 13.1 Global scatter + fitted curve (using dense X)
xx = np.linspace(model_df['log_gdp_per_capita'].min(), model_df['log_gdp_per_capita'].max(), 200).reshape(-1,1)
yy = best_pipe.predict(xx)

plt.figure(figsize=(9,6))
sns.scatterplot(x='log_gdp_per_capita', y='co2_wins', data=model_df, alpha=0.25)
plt.plot(xx, yy, linewidth=2, label=f'Poly Ridge deg={best["degree"]} alpha={best["alpha"]}')
plt.xlabel('log(GDP per capita)')
plt.ylabel('CO2 per capita (winsorized)')
plt.title('EKC - Pooled data 2000-2020 (scatter + fitted)')
plt.legend()
plt.tight_layout()
plt.savefig("outputs/ekc_scatter_fitted.png")
plt.close()

# 13.2 Time-series for top 6 green candidates (CO2 trend vs GDP)
top6 = list(green_candidates.head(6).index)
ts = pivot[pivot['Country Name'].isin(top6)].copy()
# plot CO2 and GDP in subplots
for country in top6:
    sub = ts[ts['Country Name']==country]
    fig, ax1 = plt.subplots(figsize=(8,4))
    ax1.plot(sub['Year'], sub['co2_per_capita'], marker='o', label='CO2 per cap', linestyle='-')
    ax1.set_ylabel('CO2 per capita')
    ax2 = ax1.twinx()
    ax2.plot(sub['Year'], sub['gdp_per_capita'], marker='x', color='tab:orange', label='GDP per cap')
    ax2.set_ylabel('GDP per capita (USD)')
    plt.title(f"{country} - CO2 and GDP (2000-2020)")
    fig.tight_layout()
    file = f"outputs/ts_{country.replace(' ','_')}.png"
    plt.savefig(file)
    plt.close()

# 13.3 Regional heatmap: avg CO2 by region (using wdi_country region column heuristically)
# attempt to find region column (common names: 'Region', 'Region Name', 'RegionName','RegionCode')
region_col = None
for candidate in ['Region','Region Name','RegionName','RegionCode','Region code','region']:
    if candidate in pivot.columns:
        region_col = candidate
        break

if region_col:
    reg = pivot.groupby([region_col,'Year'])['co2_per_capita'].mean().reset_index()
    # pivot to heatmap: rows region, cols year
    hp = reg.pivot(index=region_col, columns='Year', values='co2_per_capita').fillna(0)
    plt.figure(figsize=(12,6))
    sns.heatmap(hp, cmap='viridis', linewidths=.5)
    plt.title('Avg CO2 per capita by Region (2000-2020)')
    plt.tight_layout()
    plt.savefig("outputs/regional_co2_heatmap.png")
    plt.close()
    print("3 temel görsel (scatter, zaman serileri, heatmap) outputs klasörüne kaydedildi.")
else:
    print("Bölge (Region) sütunu bulunamadı; ısı haritası atlandı.")

3 temel görsel (scatter, zaman serileri, heatmap) outputs klasörüne kaydedildi.


In [36]:
# 14) Kaydet: temizlenmiş veri, model sonuçları, green candidates
pivot.to_csv("outputs/WDI_EKC_Cleaned_2000_2020.csv", index=False)
model_df.to_csv("outputs/WDI_EKC_ModelData.csv", index=False)
green_candidates.to_csv("outputs/Green_Growth_Candidates_2000_2020.csv")

# For Tableau & Power BI: save both "row-level cleaned" and aggregated summaries
# Tableau prefers denormalized row-level; PowerBI is fine with same CSV
pivot.to_csv("outputs/Tableau_WDI_EKC.csv", index=False)
pivot.to_csv("outputs/PowerBI_WDI_EKC.csv", index=False)

# Zip outputs for easy download
import zipfile
zf_name = "WDI_EKC_outputs.zip"
with zipfile.ZipFile(zf_name, 'w') as zf:
    for root, _, files_ in os.walk('outputs'):
        for f in files_:
            zf.write(os.path.join(root,f), arcname=f)
    # also include key CSVs
    zf.write("outputs/WDI_EKC_Cleaned_2000_2020.csv", arcname="WDI_EKC_Cleaned_2000_2020.csv")
    zf.write("outputs/WDI_EKC_ModelData.csv", arcname="WDI_EKC_ModelData.csv")
    zf.write("outputs/Green_Growth_Candidates_2000_2020.csv", arcname="Green_Growth_Candidates_2000_2020.csv")

print("\n✅ Pipeline tamamlandı. Üretilen dosyalar 'outputs/' klasöründe ve ZIP: ", zf_name)
files.download(zf_name)

# 15) Kısa özet çıktı (ekran)
print("\nÖzet:")
print("- Cleaned row-level CSV: outputs/WDI_EKC_Cleaned_2000_2020.csv")
print("- Model data (with predictions/residuals): outputs/WDI_EKC_ModelData.csv")
print("- Green growth candidates: outputs/Green_Growth_Candidates_2000_2020.csv")
print("- Görseller: outputs/ekc_scatter_fitted.png, outputs/regional_co2_heatmap.png, outputs/ts_<country>.png")


✅ Pipeline tamamlandı. Üretilen dosyalar 'outputs/' klasöründe ve ZIP:  WDI_EKC_outputs.zip


/usr/lib/python3.12/zipfile/__init__.py:1624: UserWarning: Duplicate name: 'WDI_EKC_Cleaned_2000_2020.csv'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/lib/python3.12/zipfile/__init__.py:1624: UserWarning: Duplicate name: 'WDI_EKC_ModelData.csv'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/lib/python3.12/zipfile/__init__.py:1624: UserWarning: Duplicate name: 'Green_Growth_Candidates_2000_2020.csv'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Özet:
- Cleaned row-level CSV: outputs/WDI_EKC_Cleaned_2000_2020.csv
- Model data (with predictions/residuals): outputs/WDI_EKC_ModelData.csv
- Green growth candidates: outputs/Green_Growth_Candidates_2000_2020.csv
- Görseller: outputs/ekc_scatter_fitted.png, outputs/regional_co2_heatmap.png, outputs/ts_<country>.png
